In [34]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [35]:
import pyspark
from pyspark.sql import SparkSession
import urllib.request

In [36]:
spark = (SparkSession.builder 
    .master("local[*]") 
    .appName('test') 
    .getOrCreate())

In [37]:
urllib.request.urlretrieve('https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz', 'fhv_tripdata_2019-10.csv.gz') 

('fhv_tripdata_2019-10.csv.gz', <http.client.HTTPMessage at 0x25aaf6a67b0>)

In [ ]:
%%sh
zcat hv_tripdata_2019-10.csv.gz | wc -l

In [38]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [40]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [42]:
%%sh 
zcat fhv_tripdata_2019-10.csv.gz | head -n 1001 > head.csv

In [43]:
import pandas as pd

In [44]:
df_pandas = pd.read_csv('head.csv')

In [45]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [47]:
spark.createDataFrame(df_pandas).printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



Integer - 4 bytes
<br>
Long - 8 bytes

In [48]:
from pyspark.sql import types

In [56]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [58]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [59]:
df = df.repartition(24)

In [60]:
df.write.parquet('04_pyspark/')

In [61]:
df = spark.read.parquet('04_pyspark/')

In [62]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [63]:
from pyspark.sql import functions as F

In [64]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|     B01467         |2019-10-06 06:28:47|2019-10-06 06:56:59|         193|          76|   NULL|       B01467         |
|              B02735|2019-10-21 16:09:25|2019-10-21 16:18:21|         264|         254|   NULL|                B02864|
|              B00856|2019-10-07 18:49:52|2019-10-07 18:55:37|         264|          61|   NULL|                B02878|
|              B00112|2019-10-26 20:24:02|2019-10-26 20:33:16|         264|          67|   NULL|                B00112|
|              B03162|2019-10-13 13:21:27|2019-10-13 14:54:12|         116|         198|   NULL|                B03162|
|              B00445|2019-10-20 09:52:3

In [65]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [71]:
crazy_stuff('B02884')

's/b44'

In [67]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [78]:
df = df.withColumn('dispatching_base_num', F.trim(df.dispatching_base_num))
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01467|2019-10-06 06:28:47|2019-10-06 06:56:59|         193|          76|   NULL|       B01467         |
|              B02735|2019-10-21 16:09:25|2019-10-21 16:18:21|         264|         254|   NULL|                B02864|
|              B00856|2019-10-07 18:49:52|2019-10-07 18:55:37|         264|          61|   NULL|                B02878|
|              B00112|2019-10-26 20:24:02|2019-10-26 20:33:16|         264|          67|   NULL|                B00112|
|              B03162|2019-10-13 13:21:27|2019-10-13 14:54:12|         116|         198|   NULL|                B03162|
|              B00445|2019-10-20 09:52:3

In [79]:
df2 = (df
.withColumn('pickup_date', F.to_date(df.pickup_datetime))
.withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) 
.withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num))
.select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID'))

In [81]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01467|2019-10-06 06:28:47|2019-10-06 06:56:59|         193|          76|   NULL|       B01467         |
|              B02735|2019-10-21 16:09:25|2019-10-21 16:18:21|         264|         254|   NULL|                B02864|
|              B00856|2019-10-07 18:49:52|2019-10-07 18:55:37|         264|          61|   NULL|                B02878|
|              B00112|2019-10-26 20:24:02|2019-10-26 20:33:16|         264|          67|   NULL|                B00112|
|              B03162|2019-10-13 13:21:27|2019-10-13 14:54:12|         116|         198|   NULL|                B03162|
|              B00445|2019-10-20 09:52:3

In [83]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.dispatching_base_num == 'B03016').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2019-10-21 08:16:35|2019-10-21 08:51:25|         264|         242|
|2019-10-16 06:12:16|2019-10-16 06:36:21|         264|         136|
|2019-10-25 14:23:19|2019-10-25 14:51:04|         264|         265|
|2019-10-23 10:30:01|2019-10-23 10:51:07|         264|         242|
|2019-10-18 08:24:23|2019-10-18 08:34:23|         264|         254|
|2019-10-15 13:39:49|2019-10-15 14:15:11|         264|          74|
|2019-10-02 11:11:33|2019-10-02 11:18:59|         264|         242|
|2019-10-27 06:54:41|2019-10-27 07:02:12|         264|         242|
|2019-10-19 10:43:05|2019-10-19 10:55:45|         264|         212|
|2019-10-02 20:33:27|2019-10-02 20:40:49|         264|          32|
|2019-10-28 14:51:31|2019-10-28 15:18:22|         264|         185|
|2019-10-12 19:58:59|2019-10-12 20:06:27|       